# Benchmark comparison of protein sequence preprocessing effect on learning task for Pfam family classification

Notebook below allows to play with parameters of the workflow and yield different results.
As a default state it is set with parameters described in the paper as well as used as a base for plots.

In [2]:
import os

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, interact_manual, fixed
import sys
from urllib import request
from scripts.data_preparation_support import data_preparation, split_to_train_test
from scripts.shorten_encoding import compress_protein_data_original, compress_protein_data_singletons, compress_protein_data_triplets
from scripts.prepare_vectors import split_data_to_classes, prepare_biovec_model, load_biovec_model_with_classes
from scripts.prepare_input_stats import input_analysis
from scripts.model_scripts.decision_trees import decision_tree_func
from scripts.model_scripts.random_tree import random_tree_func
from scripts.model_scripts.mlp import mlp_func
from scripts.model_scripts.nearest_neighbours import nearest_neighbours_func
from scripts.model_scripts.deep_learning import deep_learning_func
from time import time

## Dataset
In this project I provide already prepared - cleaned dataset used for the report, so download part below is commented out. However if desired, parameters can be tweaked to obtain altered version.

Raw data for this project is easily accessible by the Swissprot part of the Uniprot Database.
Unfortunately, full file weights around 250Mb, so instead I provide link for download.

In case server can't handle the download through Python, it is still possiblle to download tab-separated file directly
from the website: [LINK](https://www.uniprot.org/uniprot/?query=reviewed%3Ayes&columns=id%2Cdatabase(Pfam)%2Corganism%2Csequence)

It suggested, although not neccessary, to put downloaded file into ./data/full directory for clarity and ease of use of the default values below.

In [3]:
# url="https://www.uniprot.org/uniprot/?query=reviewed:yes&format=tab&columns=id,database(Pfam),organism,sequence"
# request.urlretrieve(url, "./data/full/uniprot-reviewed_yes.tab")

In [4]:
def data_prepare_widget():
    org_w = widgets.BoundedIntText(value=1000, min=1, max=5000, step=1, description="Organisms")
    fam_w = widgets.BoundedIntText(value=20, min=1, max=200, step=1, description="Families")
    infile = widgets.Text(value="./data/full/uniprot-reviewed_yes.tab",  description="Input path")
    outfile = widgets.Text(value="./data/data_file.fasta", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    widget(data_preparation, n_org=org_w, n_fam=fam_w, file_path=infile, outfile_path=outfile)

In [5]:
data_prepare_widget()

interactive(children=(Text(value='./data/full/uniprot-reviewed_yes.tab', description='Input path'), Text(value…

At this point of code raw Swissprot data was filtered by the top number of organisms and picked proteins containing single domain from top biggest families.

However, before splitting our data to train and test, we want to first adress very simmilar sequences.
It can be done using CD-HIT package.

In [6]:
def run_cdhit(input_f="./data/to_cluster/data_file.fasta", output_f="./data/to_cluster/data_file_outttt.fasta", c="0.99"):
    c = str(c)
    print("Begin CD-HIT")
    os.system(f"./CD-HIT/cd-hit -i {input_f} -o {output_f} -c {c}")


def run_cdhit_widget():
    c = widgets.BoundedFloatText(value=0.99, min=0.7, max=1, description="Simmilarity")
    infile = widgets.Text(value="./data/data_file.fasta",  description="Input path")
    outfile = widgets.Text(value="./data/clustering/data_file_clustered.fasta", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    widget(run_cdhit, c=c, input_f=infile, output_f=outfile)

In [7]:
run_cdhit_widget()

interactive(children=(Text(value='./data/data_file.fasta', description='Input path'), Text(value='./data/clust…

Now our data is cleaned out and ready to process.

## Data processing

For this project data is processed in multiple ways.

 - Standard single-letter encoding
 - Conversion into numbers and using int-8 encoding
 - Conversion into 3-mers, encoding each 3-mer as a number and using int-16 encoding
 - Using Biovec vector encoder

First we need to split our data into training and test parts.
To avoid dominance of the biggest families, training data will contain the same number of sequences for each PFAM family.

### Warning!
Please analyse the histogram first before providing number of sequences per family to train.
Families with number of representatives lower than provided will be filtered out!

Sometimes it might be beneficial to filter out couple families with low coverage.

In [8]:
def histogram_widget():
    infile = widgets.Text(value="./data/clustering/data_file_clustered.fasta",  description="Input path")
    widget = interact_manual.options(manual_name="Prepare histogram")

    widget(input_analysis, input_file=infile)

In [9]:
histogram_widget()

interactive(children=(Text(value='./data/clustering/data_file_clustered.fasta', description='Input path'), But…

In [10]:
def split_to_train_test_widget():
    train_val = widgets.BoundedIntText(value=650, min=1, max=5000, step=1, description="N. to train")
    infile = widgets.Text(value="./data/clustering/data_file_clustered.fasta",  description="Input path")
    outfile = widgets.Text(value="./data/clean_dataset.pkl", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    new_val = widget(split_to_train_test, train_val=train_val, infile=infile, outfile=outfile)
    return new_val

In [11]:
train_val_global = split_to_train_test_widget()

interactive(children=(BoundedIntText(value=750, description='N. to train', max=5000, min=1), Text(value='./dat…

This process will save data into convenient pickle object. This way instead of keeping 4 separate files or trying to split our data in one file we can easily load a list prepared to use.

In [12]:
compress_protein_data_original(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_original.pkl')

In [13]:
compress_protein_data_singletons(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_singletons.pkl')

In [14]:
compress_protein_data_triplets(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_triplets.pkl')

## Biovec

Biovec model is built on top of the original implementation, thus data handling must be a bit different, to fit authors requirements.
Sequences must be split into class fasta files, and one combined with identical names.
Combined file will be used to generate model, wchich will be then saved.
This saved model, is next loaded again, but this time, we also provide family information.

After this procedure we are finally left with a data ready to split into training and test, perform learning and predictions.


### Warning !
This time widget is not provided, because of the ./data/vectors/combined_corpus.fasta file.
If Biovec recognizes this file it will skip creating a new model.
That's why paths here are fixed to ensure, old corpus file is deleted.

### Warning !
It might happen, that during loading process there will be information, that model did not train on certain triplets.
It is connected with fragments containing extended alphabet like "X" and are ignored.

In [15]:
split_data_to_classes(infile="./data/clean_dataset.pkl", output_folder="./data/vectors/class_folder",
                          output_combined_file="./data/vectors/combined.fasta")

In [16]:
prepare_biovec_model(infile="./data/vectors/combined.fasta", outfile="./data/vectors/ProtVec_model.model")

In [17]:
def load_biovec_widget():
    train_val = widgets.BoundedIntText(value=650, min=1, max=5000, step=1, description="N. to train")

    widget = interact_manual.options(manual_name="Prepare data")

    new_val = widget(load_biovec_model_with_classes, train_size=train_val, input_model=fixed("./data/vectors/ProtVec_model.model"), class_folder=fixed("./data/vectors/class_folder"), outfile=fixed('./data/clean_dataset_biovec.pkl'))

In [18]:
load_biovec_widget()

interactive(children=(BoundedIntText(value=750, description='N. to train', max=5000, min=1), Button(descriptio…

## Evaluation

Now, that we have all data prepared, we can evaluate them both in accuracy and runtime.

Several models will be created - please note, that not all of them are equally suitable for this kind of data - the point is in efficiency comparison.

- Decision trees
- Random trees
- Nearest Neighbours
- MLP
- Simple Machine Learning with Dense layers

In [8]:
s1 = time()
acc1 = decision_tree_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2 = decision_tree_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3 = decision_tree_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4 = decision_tree_func("./data/clean_dataset_biovec.pkl")
t4 = time() - s4

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 20
----- Model accuracy: 0.145
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 60
----- Model accuracy: 0.724
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 90
----- Model accuracy: 0.642
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 80
----- Model accuracy: 0.727


In [9]:
print(f"Statistic | Original | Singletons | Triplets | Biovec")
print(f"----------|----------|------------|----------|---------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t")

Statistic | Original | Singletons | Triplets | Biovec
----------|----------|------------|----------|---------
Time      | 12.59 	 | 10.14 	 | 6.47 	 | 0.31 	
Accuracy  | 0.1448 	 | 0.724 	 | 0.6417 	 | 0.7275 	


In [35]:
s1 = time()
acc1 = random_tree_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2 = random_tree_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3 = random_tree_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4 = random_tree_func("./data/clean_dataset_biovec.pkl")
t4 = time() - s4

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)

Fitting 2 folds for each of 15 candidates, totalling 30 fits
Best model:
- random_state: 1234
- max_depth: 20
- max_features: 20
----- Model accuracy: 0.408
Fitting 2 folds for each of 15 candidates, totalling 30 fits
Best model:
- random_state: 1234
- max_depth: 40
- max_features: 20
----- Model accuracy: 0.794
Fitting 2 folds for each of 15 candidates, totalling 30 fits
Best model:
- random_state: 1234
- max_depth: 20
- max_features: 20
----- Model accuracy: 0.761
Fitting 2 folds for each of 15 candidates, totalling 30 fits


/home/exsto/anaconda3/envs/biovec_new/lib/python3.8/site-packages/sklearn/model_selection/_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best model:
- random_state: 1234
- max_depth: 20
- max_features: 20
----- Model accuracy: 0.059


5.68511

In [36]:
print(f"Statistic | Original | Singletons | Triplets | Biovec")
print(f"----------|----------|------------|----------|---------")
print(f"Time      | {t1}\t | {t2}\t | {t3} \t | {t4} \t")
print(f"Accuracy  | {acc1}\t | {acc2}\t | {acc3} \t | {acc4} \t")

Statistic | Original | Singletons | Triplets | Biovec
----------|----------|------------|----------|---------
Time      | 43.41687726974487	 | 35.38896465301514	 | 40.851250410079956 	 | 5.685107231140137 	
Accuracy  | 0.4076	 | 0.7941	 | 0.7605 	 | 0.0588 	


In [ ]:
s1 = time()
acc1 = nearest_neighbours_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2 = nearest_neighbours_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3 = nearest_neighbours_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4 = nearest_neighbours_func("./data/clean_dataset_biovec.pkl")
t4 = time() - s4

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)

In [ ]:
print(f"Statistic | Original | Singletons | Triplets | Biovec")
print(f"----------|----------|------------|----------|---------")
print(f"Time      | {t1}\t | {t2}\t | {t3} \t | {t4} \t")
print(f"Accuracy  | {acc1}\t | {acc2}\t | {acc3} \t | {acc4} \t")

In [ ]:
s1 = time()
acc1 = mlp_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2 = mlp_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3 = mlp_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4 = mlp_func("./data/clean_dataset_biovec.pkl")
t4 = time() - s4

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)

In [ ]:
print(f"Statistic | Original | Singletons | Triplets | Biovec")
print(f"----------|----------|------------|----------|---------")
print(f"Time      | {t1}\t | {t2}\t | {t3} \t | {t4} \t")
print(f"Accuracy  | {acc1}\t | {acc2}\t | {acc3} \t | {acc4} \t")

In [ ]:
s1 = time()
acc1 = deep_learning_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2 = deep_learning_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3 = deep_learning_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4 = deep_learning_func("./data/clean_dataset_biovec.pkl")
t4 = time() - s4

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)

In [ ]:
print(f"Statistic | Original | Singletons | Triplets | Biovec")
print(f"----------|----------|------------|----------|---------")
print(f"Time      | {t1}\t | {t2}\t | {t3} \t | {t4} \t")
print(f"Accuracy  | {acc1}\t | {acc2}\t | {acc3} \t | {acc4} \t")